# Numba 0.55.0 Release Demo

This notebook contains a demonstration of new features present in the 0.55.x release of Numba. Whilst release notes are produced as part of the [CHANGE_LOG](https://github.com/numba/numba/blob/release0.55/CHANGE_LOG), there's nothing like seeing code in action!

This release contains updated NumPy and Python version support, some newly supported NumPy functions, a lot of improvements to native debugging and some internal refactoring surrounding error handling. In this notebook, the new CPU target features are demonstrated. The [CUDA target](https://numba.readthedocs.io/en/stable/cuda-reference/index.html) also gained a lot of new features in 0.55.0 and [@gmarkall](https://github.com/gmarkall) has created a [demo notebook](https://mybinder.org/v2/gh/numba/numba-examples/master?filepath=notebooks%2FNumba_055_CUDA_Release_Demo.ipynb) especially for these!

Key internal/support changes in this release are:

- Python 3.10 is now supported.
- NumPy support extended to include 1.21.x and is now restricted to (1.18 <= x < 1.22).

Intel also kindly sponsored research and development into native debug (DWARF)
support and handling per-function compilation flags (largely demonstrated in [0.55 debug enhancements demo](TODO)):

* Line number/location tracking is much improved.
* Numba's internal representation of containers (e.g. tuples, arrays) are now
  encoded as structures.
* Numba's per-function compilation flags are encoded into the ABI field of the
  mangled name of the function such that it's possible to compile and
  differentiate between versions of the same function with different flags set.

## Python support enhancements

In [ ]:
from numba import njit
import numba
assert numba.version_info.short >= (0, 55)
from numba.core import types
import numpy as np

Numba 0.55 gains experimental support for `isinstance` ([@guilhermeleobas](https://github.com/guilhermeleobas) and [@stuartarchibald](https://github.com/stuartarchibald)). A quick demonstration...

In [ ]:
from numba import typed, float64
from numba.core.errors import NumbaWarning

# isinstance is experimental, it will display a warning stating this, for demo purposes
# suppress this warning:
import warnings
warnings.filterwarnings("ignore", category=NumbaWarning)

def demo_isinstance(x_inst):
    xtype = f'type: {type(x_inst)}, value: "{x_inst}"'
    @njit
    def impl(x):
        print('Testing:', xtype)
        def answer(clazz):
            return "yes" if isinstance(x, clazz) else "no"

        print("Is ", x, "a string?...", answer(str))
        print("Is ", x, "an int?  ...", answer(int))
        print("Is ", x, "a float? ...", answer(float))
        print("Is ", x, "complex? ...", answer(complex))
        print("Is ", x, "a tuple? ...", answer(tuple))
        print("Is ", x, "a set?   ...", answer(set))
        print("Is ", x, "a list?   ...", answer(list))
        print("")
    impl(x_inst)

for x in ('a string', # string
          1, # int
          2.3, # float
          4j, # complex
          np.zeros(3), # np array
          typed.Dict.empty(float64, float64), # typed dict
          (1, 2, 3), # tuple
          {4, 5, 6}, # set
          [7, 8, 9] # list
         ):
    demo_isinstance(x)


The `isinstance` implementation also works with a subset of Numba types, for example:

In [ ]:
from numba import float64, intp

@njit
def demo_isinstance_nb_types(x):
    print('Testing value:', x)
    def answer(clazz):
        return "yes" if isinstance(x, clazz) else "no"
    
    print("Is ", x, "a float64 type?...", answer(float64))
    print("Is ", x, "an intp type?  ...", answer(intp))
    print("")

for x in (1, # int
          2.3, # float
         ):
    demo_isinstance_nb_types(x)


## NumPy support enhancements

Numba 0.55 contains support for some more NumPy functions:

* ``np.broadcast_to`` ([@guilhermeleobas](https://github.com/guilhermeleobas))
* ``np.float_power`` ([@guilhermeleobas](https://github.com/guilhermeleobas))
* ``np.cbrt`` ([@guilhermeleobas](https://github.com/guilhermeleobas))
* ``np.logspace`` ([@guoqiangqi](https://github.com/guoqiangqi))
* ``np.take_along_axis`` ([@itamarst](https://github.com/itamarst))
* ``np.average`` ([@hadia206](https://github.com/hadia206) and [@slnguyen](https://github.com/slnguyen))

and also enhances the support for:

* ``np.argmin`` gains support for the ``axis`` kwarg ([@itamarst](https://github.com/itamarst)).
* ``np.ndarray.astype`` gains support for types expressed as literal strings ([@njriasan](https://github.com/njriasan)).
* `__setitem__` and `__getitem__` works for arrays in structured dtypes ([@luk-f-a](https://github.com/luk-f-a) and [@gmarkall](https://github.com/gmarkall)).

In [ ]:
a = np.arange(24).reshape((3, 1, 4, 2))
ai = np.argsort(a)

@njit
def demo_numpy_new_features():
    print("np.broadcast_to\n", np.broadcast_to(np.arange(3), (2, 3)),)
    print("np.float_power\n", np.float_power(np.arange(5), np.arange(5)))
    print("np.cbrt\n", np.cbrt(np.arange(0, 28, 3)))
    print("np.logspace\n", np.logspace(0, 10, 4))
    print("np.take_along_axis\n", np.take_along_axis(a, ai, axis=1))
    print("np.average\n", np.average(np.arange(10.)))

demo_numpy_new_features()

Enhancements:

In [ ]:
# define a structured dtype for use in the demo
rec_ty = np.dtype([('i', np.int32), ('j', np.float32, (3, 2))])

@njit
def demo_numpy_enhancements():
    # Axis support in np.argmin
    x = np.arange(12).reshape((3, 2, 2))
    x[:, 1, :] = -10
    print("np.argmin(x, axis=2)\n", np.argmin(x, axis=2))
    
    # Literal string types in np.ndarray.astype
    x = np.arange(10)
    print("np.ndarray.astype('float64')\n", x.astype('float64'))
    print("np.ndarray.astype('complex64')\n", x.astype('complex64'))

    # Assigning to a record array
    arr = np.zeros(1, dtype=rec_ty)
    rec = arr[0]
    rec['j'][:, :] = np.ones((3, 2))
    print("Array assignment with a structured dtype\n", arr)
    
    
demo_numpy_enhancements()

----

## See Also

- [0.55 CUDA Demo](https://mybinder.org/v2/gh/numba/numba-examples/master?filepath=notebooks%2FNumba_055_CUDA_Release_Demo.ipynb)
- [0.55 debug enhancements demo](TODO)